# Project

In [7]:
using CSV, DataFrames, JuMP, Gurobi, Plots, LinearAlgebra, Dates

In [8]:
fires = CSV.read("data/wildfire_suppression.csv", DataFrame, header=true);
first(fires, 5)

Row,report_id,fire_id,area,date,longitude,latitude,from_date,total_crews_sent
,Int64,Int64,Float64?,String31,Float64?,Float64?,String31,Float64
1,2714023,2714022,150.0,2015-05-06 10:15:00,-71.2519,43.7811,2015-05-05 09:30:00,45.0
2,2714037,2714022,275.0,2015-05-08 00:30:00,-71.2519,43.7811,2015-05-07 11:00:00,45.0
3,2714050,2714022,275.0,2015-05-09 00:30:00,-71.2519,43.7811,2015-05-08 11:00:00,33.0
4,2714066,2714022,275.0,2015-05-10 13:00:00,-71.2519,43.7811,2015-05-09 15:00:00,0.0
5,2714082,2714081,205.0,2015-05-07 07:15:00,-67.0125,44.7917,2015-05-06 20:30:00,21.0


### Preprocessing

In [9]:
println("Number of rows: ", size(fires, 1))
fires = filter(row -> !("missing" in string.(collect(row))), fires);
println("Number of rows after removing missing values: ", size(fires, 1))

dates = []
for i in 1:nrow(fires)
    res = DateTime.(fires[i, :date][1:10], "yyyy-mm-dd")
    push!(dates, res)
end
fires[!, :date] = dates

fires_unique = combine(groupby(fires, :fire_id)) do group
    idx_max = argmax(group.total_crews_sent)
    return group[idx_max, :]
end;

# Sort by date
sorted_fires = sort(fires_unique, :date);

Number of rows: 38505
Number of rows after removing missing values: 38491


In [10]:
sorted_fires = sort(fires_unique, :date);

In [5]:
fires_df = deepcopy(sorted_fires)
first(fires_df, 6)

Row,fire_id,report_id,area,date,longitude,latitude,from_date,total_crews_sent
,Int64,Int64,Float64?,Any,Float64?,Float64?,String31,Float64
1,2833110,2833133,1.0,2014-10-20T00:00:00,-89.7867,32.6819,2014-10-20 12:15:00,6.0
2,2847976,2847977,500.0,2014-12-24T00:00:00,-114.446,32.8555,2014-12-24 01:00:00,48.0
3,2836444,2836445,4.5,2015-01-06T00:00:00,-86.7161,30.8203,2015-01-06 11:45:00,15.0
4,2850948,2850982,200.0,2015-01-09T00:00:00,-81.4789,26.0269,2015-01-09 10:00:00,2.0
5,2851007,2851018,1.0,2015-01-12T00:00:00,-87.0697,30.9225,2015-01-12 14:15:00,1.0
6,2730867,2730879,2.0,2015-01-17T00:00:00,-89.7192,31.9842,2015-01-17 14:30:00,13.0


In [ ]:
day = 2015-08-01

In [6]:
Fires = Matrix(fires_df);
n_fires = 1:nrow(fires_df);
Surfaces = fires_df[:, :area];


### Distance feature

In [12]:
# Calculate Euclidian distance
function euclidean_distance(lat1, lon1, lat2, lon2)
    return sqrt((lat2 - lat1)^2 + (lon2 - lon1)^2)
end

# Calculate the Haversine distance
function haversine_distance(lat1, lon1, lat2, lon2)
    R = 6371.0 # Earth radius in kilometers
    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    lat1 = deg2rad(lat1)
    lat2 = deg2rad(lat2)

    a = sin(dLat/2)^2 + cos(lat1) * cos(lat2) * sin(dLon/2)^2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c
end

# Returns distance matrix of distances between fires
function distance_matrix(df, euclidian=1)
    n = nrow(df)
    dist_matrix = zeros(n, n)
    for i in 1:n
        for j in (i+1):n
            if euclidian == 1
                dist = euclidean_distance(df[i, :latitude], df[i, :longitude], df[j, :latitude], df[j, :longitude])
            else
                dist = haversine_distance(df[i, :latitude], df[i, :longitude], df[j, :latitude], df[j, :longitude])
            end
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist
        end
    end
    return dist_matrix
end;

Distances = distance_matrix(fires_df)

5892×5892 Matrix{Float64}:
  0.0       24.6595   3.59084  10.6446   …  11.1891    8.01214    6.27449
 24.6595     0.0     27.8041   33.6665      15.4125   18.1237    19.8254
  3.59084   27.8041   0.0       7.09962     13.4905   11.6023     9.8601
 10.6446    33.6665   7.09962   0.0         18.4828   18.6033    16.9114
  3.23689   27.444    0.36809   7.43128     13.1694   11.2459     9.50983
  0.701035  24.7418   3.22071  10.1681   …  10.9676    8.43817    6.75195
 11.6924    34.7385   8.12938   1.08922     19.5299   19.6667    17.9643
  5.34969   19.8794   8.92409  15.8796       8.74839   2.74344    1.44226
  3.18041   21.8901   6.76854  13.7823       9.64945   4.83478    3.14342
  3.17966   21.8931   6.76792  13.7822       9.65391   4.83529    3.14305
  ⋮                                      ⋱             ⋮         
  7.46794   18.5118  11.0579   18.0586       9.2725    0.545039   1.35618
  6.25792   18.8302   9.79606  16.6682       8.03603   2.22005    1.76538
  2.24908   25.9862   1

### First model

In [13]:
n_fighters = 1:10 # to change
n_days = 1:3

1:3

In [ ]:
a routing model that ignores disparities in fire sizes; and

In [14]:
model = Model(Gurobi.Optimizer);

@variable(model, X[n_fighters, n_days, n_fires], binary = true);
@variable(model, Y[n_fighters, n_days, n_fires, n_fires], binary = true);

@objective(model, Min, sum(sum(sum(X[i,t,k] * Surfaces[k] for i in n_fighters) for k in n_fires) for t in n_days) +
                        sum(sum(sum(sum(Y[i,t,k1,k2] * Distances[k1,k2] for i in n_fighters) for k1 in n_fires) for k2 in n_fires) for t in n_days));


@constraint(model, constraint_fighters1[i in n_fighters], sum(X[i,t,k] for k in n_fires) <= 1);
@constraint(model, constraint_resources1[k in n_fires], sum(X1[i,t,k] for i in n_fighters) <= Gamma(S[k]));
@constraint(model, constraint_resources2[k in n_fires], sum(X2[i,t,k] for i in n_fighters) <= Gamma(S[k]));
@constraint(model, constraint_resources3[k in n_fires], sum(X3[i,t,k] for i in n_fighters) <= Gamma(S[k]));

optimize!(model);